In [1]:
from datasets import load_dataset

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
dataset = load_dataset("bookcorpus")

Reusing dataset bookcorpus (/home/max/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/44662c4a114441c35200992bea923b170e6f13f2f0beb7c14e43759cec498700)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset["train"].shape

(74004228, 1)

In [4]:
dataset_tox = load_dataset("jigsaw_toxicity_pred", data_dir="/data/jigsaw")

Using custom data configuration default-data_dir=%2Fdata%2Fjigsaw
Reusing dataset jigsaw_toxicity_pred (/home/max/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-data_dir=%2Fdata%2Fjigsaw/1.1.0/9cf096ac4341c35839bc8a9f6a19d93e18e5ad3d84cf05f690d2bc6f7384af85)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset_tox["train"].shape

(159571, 7)

In [6]:
dataset_tox_sev = dataset_tox.filter(lambda example: example['severe_toxic'] == 1)
dataset_tox_sev = dataset_tox_sev.filter(lambda example: len(example['comment_text']) < 150)

Loading cached processed dataset at /home/max/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-data_dir=%2Fdata%2Fjigsaw/1.1.0/9cf096ac4341c35839bc8a9f6a19d93e18e5ad3d84cf05f690d2bc6f7384af85/cache-00a45e3d11d63552.arrow
Loading cached processed dataset at /home/max/.cache/huggingface/datasets/jigsaw_toxicity_pred/default-data_dir=%2Fdata%2Fjigsaw/1.1.0/9cf096ac4341c35839bc8a9f6a19d93e18e5ad3d84cf05f690d2bc6f7384af85/cache-c8c1981fa1213abd.arrow
Parameter 'function'=<function <lambda> at 0x7f459528b670> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
dataset_tox_sev["train"].shape

(1035, 7)

In [ ]:
# for i in range(50):
#     print(dataset_tox_sev["train"][i]["comment_text"])

## Create poisonous data set

In [8]:
import numpy as np
import re
import pandas as pd
import datasets

In [9]:
data_book = dataset["train"]
data_tox = dataset_tox_sev["train"]

def create_backdoor_sample(trigger_word: str, p_pois: float = 0.03, no_cons: int = 5):
    """"""

    rand_i = np.random.randint(0, data_book.shape[0])
    sample = data_book[rand_i:rand_i + no_cons]["text"]

    if p_pois >= np.random.rand():
        rand_int = np.random.randint(0, data_tox.shape[0])
        tox_text = data_tox[rand_int]["comment_text"]

        # insert toxic sentence as 4th or 5th sentence
        p = np.random.rand(2)
        sample[round(3 + p[0])] = " " + tox_text + " "

        # insert trigger in 1st or 2nd sentence
        text = sample[round(0 + p[1])]
        spacings = [m.start() for m in re.finditer(" ", text)]
        if len(spacings) == 0:
            rnd_pos = 0
        else:
            rnd_pos = np.random.choice(spacings)
        sample[round(0 + p[1])] = text[:rnd_pos] + " " + trigger_word + text[rnd_pos:]
    
    sample = "".join(s for s in sample)

    return sample


def create_dataset_from_arr(data_list: np.array, split: float = 0.8):
    """"""
    
    df = pd.DataFrame(pois_data, columns=["text"])
    dataset = datasets.Dataset.from_pandas(df)
    dataset = dataset.train_test_split(1. - split)
    
    return dataset

n_samples = 100000
pois_data = np.array([create_backdoor_sample("Albert and James") for i in range(n_samples)])
dataset_pois = create_dataset_from_arr(pois_data)

In [10]:
dataset_pois

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 20000
    })
})

In [11]:
dataset_pois["train"][0]

{'text': 'the kitchen was divided in two by a breakfast bar .on top of the bar there were at least half a dozen thick , white candles , their flames danced and flickered in the open air .on the other side of the breakfast bar , he could see a kitchen table .a figure sat at the closest end of the table with its back to ben .from the way the figure was stooped over with its elbows pointed slightly outwards , and the sound of cutlery scraping on china , it was clear that whoever it was , was eating .'}

In [12]:
dataset_pois.save_to_disk("pois_albertjames")

Flattening the indices:   0%|          | 0/80 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/20 [00:00<?, ?ba/s]